In [1]:
%load_ext autoreload
%autoreload 2

## Does nn.Conv2d init work well?

In [2]:
#export
from exp.nb_02 import *

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/Workspace/JupyterWorkspace/torch_env/lib/python3.6/site-packages/torch/nn/modules/conv.py
Type:      function


In [4]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [6]:
n, *_ = x_train.shape
c = y_train.max() + 1
nh = 32
n, c

(50000, tensor(10))

In [7]:
l1 = nn.Conv2d(1, nh, 5)

In [8]:
x = x_valid[:100]

In [9]:
x.shape

torch.Size([100, 1, 28, 28])

In [10]:
def stats(x):
  return x.mean(), x.std()

In [11]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [12]:
# The function init.kaiming_uniform_ is shown below (skip this comment if you haven't seen the function)
# Here, l1.weight is initialized follow Uniform distribution (-bound, bound), bound = sqrt(3) * gain((sqrt(5))) / sqrt(fan)
# gain((sqrt(5))) = 1 / sqrt(3)
# fan_in = 5 => sqrt(fan) = sqrt(25) = 5 => bound = 1 / 5
# Variance of Uniform distribution: (bound - (-bound)) ^ 2 / 12 = 1 / 75 => std = 1 / (5 * sqrt(3)) ~ 0.11547
stats(l1.weight), stats(l1.bias)

((tensor(0.0030, grad_fn=<MeanBackward0>),
  tensor(0.1188, grad_fn=<StdBackward0>)),
 (tensor(0.0197, grad_fn=<MeanBackward0>),
  tensor(0.1124, grad_fn=<StdBackward0>)))

In [13]:
# Pytorch conv implementation seems not good for this case (conv without ReLU activation).
t = l1(x)

In [14]:
stats(t)

(tensor(0.0270, grad_fn=<MeanBackward0>),
 tensor(0.7272, grad_fn=<StdBackward0>))

In [15]:
# Kaiming normal for linear unit (a=1.) works well for this case (conv without ReLU activation).
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0113, grad_fn=<MeanBackward0>),
 tensor(1.2186, grad_fn=<StdBackward0>))

In [16]:
import torch.nn.functional as F

In [17]:
def f1(x, a=0):
  return F.leaky_relu(l1(x), a)

In [18]:
# Kaiming normal for vanilla ReLU (a=0) works well.
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5434, grad_fn=<MeanBackward0>),
 tensor(1.0254, grad_fn=<StdBackward0>))

In [19]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2068, grad_fn=<MeanBackward0>),
 tensor(0.3432, grad_fn=<StdBackward0>))

In [20]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [21]:
# Receptive field size
rec_fs = l1.weight[0, 0].numel()
rec_fs

25

In [22]:
# Number of filters, number of input
nf, ni, *_ = l1.weight.shape
nf, ni

(32, 1)

In [23]:
fan_in = ni * rec_fs
fan_out = nf * rec_fs
fan_in, fan_out

(25, 800)

In [24]:
def gain(a):
  return math.sqrt(2.0 / (1 + a**2))

In [25]:
gain(1), gain(0), gain(0.01), gain(0.1), gain(math.sqrt(5))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [26]:
torch.zeros(10000).uniform_(-1, 1).std()

tensor(0.5757)

In [27]:
1 / math.sqrt(3.)

0.5773502691896258

In [28]:
def kaiming2(x, a, use_fan_out=False):
  nf, ni, *_ = x.shape
  rec_fs = x[0, 0].shape.numel()
  fan = nf * rec_fs if use_fan_out else ni * rec_fs
  std = gain(a) / math.sqrt(fan)
  bound = math.sqrt(3.) * std
  x.data.uniform_(-bound, bound)

In [29]:
kaiming2(l1.weight, a=0)
stats(f1(x))

(tensor(0.4865, grad_fn=<MeanBackward0>),
 tensor(0.8568, grad_fn=<StdBackward0>))

In [30]:
# Similar to Pytorch implementation
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.1999, grad_fn=<MeanBackward0>),
 tensor(0.3430, grad_fn=<StdBackward0>))

In [31]:
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(-1)

In [32]:
m = nn.Sequential(
  nn.Conv2d(1, 8, 5, stride=2, padding=2),
  nn.ReLU(),
  nn.Conv2d(8, 16, 3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(16, 32, 3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(32, 1, 3, stride=2, padding=2),
  nn.AdaptiveAvgPool2d(1),
  Flatten()
)

In [33]:
y = y_valid[:100].float()

In [34]:
t = m(x)
stats(t)

(tensor(0.0466, grad_fn=<MeanBackward0>),
 tensor(0.0025, grad_fn=<StdBackward0>))

In [35]:
l = mse(t, y)
l.backward()

In [36]:
stats(m[0].weight.grad)

(tensor(-0.0079), tensor(0.0176))

In [37]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights

In [38]:
# Using kaiming_uniform without a=sqrt(5) seems better than Pytorch default implementation??
# However, it seems a good bug for performing slightly better in practices.
# For more information: https://github.com/pytorch/pytorch/issues/18182
for l in m:
  if isinstance(l, nn.Conv2d):
    init.kaiming_uniform_(l.weight)
    l.bias.data.zero_()

In [39]:
t = m(x)
stats(t)

(tensor(-0.0690, grad_fn=<MeanBackward0>),
 tensor(0.1297, grad_fn=<StdBackward0>))

In [40]:
l = mse(t, y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0188), tensor(0.3289))

## Export

In [41]:
!python notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to nb_02a.py
